# Llama2 파인튜닝

[kullm-v2](https://huggingface.co/datasets/nlpai-lab/kullm-v2) 데이터셋으로 Llama2 모델을 QLoRA fine-tuning하여 [KoLlama2](https://huggingface.co/psymon/KoLlama2-7b)를 만드는 예제입니다.

Created by [psymon](https://github.com/psymon-dev/KoLlama2) Based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da), databricks [GitHub](https://github.com/databricks/databricks-ml-examples/blob/master/llm-models/llamav2/llamav2-7b/06_fine_tune_qlora.py) and Guardrail ML [noteBook](https://colab.research.google.com/drive/134o_cXcMe_lsvl15ZE_4Y75Kstepsntu?usp=sharing)

### 1. 필요한 파일을 설치합니다.

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 guardrail-ml==0.0.12 tensorboard
!apt-get -qq install poppler-utils tesseract-ocr
!pip install -q unstructured["local-inference"]==0.7.4 pillow==9.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/original_output.csv')

In [ ]:
# from guardrail.client import (
#     run_metrics,
#     run_simple_metrics,
#     create_dataset)

In [ ]:
df.tail()

,instruction,output
188438,"꽤 효과가 좋은 백신이구나. 맞고 싶은데, 혹시 자궁경부암 백신을 맞을 수 있는 사...","응, 여성의 경우 만 9세부터 45세까지, 남성의 경우 만 9세부터 26세가 접종대..."
188439,"아, 대상이 따로 정해져 있었구나. 잠깐만, 그럼 정부가 백신을 지원하겠다는 이야기...",그건 가다실9 백신 가격이 다시 오르면 소비자 부담이 가중되기 때문이야. 현재 필수...
188440,"아, 소비자인 국민에게 부담이 늘기 때문이구나. 그런데 왜 사람들은 가다실9가를 고...",가다실9가 백신이 가장 많은 바이러스를 예방할 수 있기 때문이야. 그래서 가다실 2...
188441,"아하, 그런 이유 때문이었구나. 정부가 곧 백신 지원을 확대한다면, 가격이 쌀 때 ...","일단 정부가 공약한 건, 가다실9 백신 접종 권장 대상자 전체에게 보험 혜택을 적용..."
188442,"그럼 꼭 기다릴 필요는 없을 수도 있겠다. 오늘 알찬 정보 많이 얻어가는데, 무척 ...","응, 그걸 바탕으로 좋은 결정을 하길 바라."


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2. 학습에 필요한 옵션들을 설정합니다.

In [ ]:
# local_rank = -1
# device_map = {"": 0}

# # 학습에 사용할 베이스 모델 경로
# model_name = "beomi/KoAlpaca-Polyglot-12.8B" # llama-2-7b 모델을 샤딩한 버전입니다. 다른 llama-2 파일을 사용해도 무관합니다.

# # 학습에 사용할 데이터셋 경로
# dataset_name = "nlpai-lab/kullm-v2"

# # 모델 체크포인트 저장 경로
# output_dir = "./results"

# # 4bit QLoRA 학습을 위한 설정
# bnb_4bit_compute_dtype = "bfloat16" # 코랩 무료버전에서 실행 시 "float16"를 사용하세요
# bnb_4bit_quant_type = "nf4"
# use_4bit = True
# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=False,
# )

# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
#         print("=" * 80)


### 3. 모델 불러오기

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/KoAlpaca-Polyglot-12.8B"  # safetensors 컨버팅된 레포
#model_id = "hhs8746/qlora-koalpaca-polyglot-12.8b-book-recommend"
#model_id = 'lcw99/polyglot-ko-12.8b-chang-instruct-chat' #'nlpai-lab/kullm-polyglot-12.8b-v2'  # 'davidkim205/komt-Llama-2-7b-chat-hf' #'lcw99/polyglot-ko-12.8b-chang-instruct-chat'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 52428800 || all params: 6653788160 || trainable%: 0.7879541509178435


In [ ]:
import pandas as pd

In [ ]:
#data = pd.read_csv('drive/MyDrive/new_formatted_prompt_dataset.csv')
data = df

In [ ]:
from datasets import Dataset, DatasetDict

# Convert the DataFrame to a Dataset
formatted_dataset = Dataset.from_pandas(data.rename(columns={'answer': 'output'}))

# Create a DatasetDict
dataset_dict = DatasetDict({"train": formatted_dataset})

dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 188443
    })
})

In [ ]:
data = dataset_dict.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}</끝>" }
)

Map:   0%|          | 0/188443 [00:00<?, ? examples/s]

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/188443 [00:00<?, ? examples/s]

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=64,
        gradient_accumulation_steps=4,
        #max_steps=100,
        num_train_epochs = 1,
        #epoch=1,## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="adamw_torch"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
10,3.400700
20,2.601000
30,2.343000
40,2.240000
50,2.183500
60,2.161300
70,2.137500
80,2.109200


Step,Training Loss
10,3.400700
20,2.601000
30,2.343000
40,2.240000
50,2.183500
60,2.161300
70,2.137500
80,2.109200


In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to(0),
        max_new_tokens=200,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [ ]:
gen('''너는 책에 대해서 답해주는 도서 챗봇이야.
나의 질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호한 질문은 모른다고 대답해.

문서 = 독서 및 정보매체의 이용 장르에서 인기 있는 도서는 공부머리 독서법:실현 가능하고 지속 가능한 독서교육의 모든 것, 아홉 살 독서 수업:부모가 알아야 할 초등 저학년 독서의 모든 것, (공부 천재가 알려 주는) 평생 도움 초등 독서법, (상위 1% 아이가 하고 있는)서울대 아빠식 문해력 독서법:독서법부터 다른 영재원 과학고 아이들의 비밀, (어린이를 위한) 진짜 공부머리 독서법, 기적의 초등 독서법, 초등공부, 독서로 시작해 글쓰기로 끝내라, 그 집 아들 독서법:사교육 중심지 대치동에서 '독서'로 살아남은 브릭 독서의 비밀, 다시, 초등 고전읽기 혁명, (초등 3, 4학년에 시작하는)하루 20분 초등 고전 읽기:공부머리, 사고머리를 키우는 진짜 독서 습관  입니다. | 시 장르에서 인기 있는 도서는 (최승호 시인의) 말놀이 동시집, 내 마음의 동시, 어린이 마음 시툰, 쉬는 시간에 똥 싸기 싫어:김개미 동시집, (박성우 시인의) 첫말 잇기 동시집, 우산 쓴 지렁이, 라면 맛있게 먹는 법, 내 입은 불량 입, 짝 바꾸는 날:이일숙 동시조, 글자동물원:이안 시 입니다.
질문 = 정보도서 추천해줘
''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 너는 책에 대해서 답해주는 도서 챗봇이야.
나의 질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호한 질문은 모른다고 대답해.

문서 = 독서 및 정보매체의 이용 장르에서 인기 있는 도서는 공부머리 독서법:실현 가능하고 지속 가능한 독서교육의 모든 것, 아홉 살 독서 수업:부모가 알아야 할 초등 저학년 독서의 모든 것, (공부 천재가 알려 주는) 평생 도움 초등 독서법, (상위 1% 아이가 하고 있는)서울대 아빠식 문해력 독서법:독서법부터 다른 영재원 과학고 아이들의 비밀, (어린이를 위한) 진짜 공부머리 독서법, 기적의 초등 독서법, 초등공부, 독서로 시작해 글쓰기로 끝내라, 그 집 아들 독서법:사교육 중심지 대치동에서 '독서'로 살아남은 브릭 독서의 비밀, 다시, 초등 고전읽기 혁명, (초등 3, 4학년에 시작하는)하루 20분 초등 고전 읽기:공부머리, 사고머리를 키우는 진짜 독서 습관  입니다. | 시 장르에서 인기 있는 도서는 (최승호 시인의) 말놀이 동시집, 내 마음의 동시, 어린이 마음 시툰, 쉬는 시간에 똥 싸기 싫어:김개미 동시집, (박성우 시인의) 첫말 잇기 동시집, 우산 쓴 지렁이, 라면 맛있게 먹는 법, 내 입은 불량 입, 짝 바꾸는 날:이일숙 동시조, 글자동물원:이안 시 입니다.
질문 = 정보도서 추천해줘


### 답변: 독서 및 정보매체의 이용 장르에서 인기 있는 도서는 공부머리 독서법:실현 가능하고 지속 가능한 독서교육의 모든 것, 아홉 살 독서 수업:부모가 알아야 할 초등 저학년 독서의 모든 것, (공부 천재가 알려주는) 평생 도움 초등 독서


In [ ]:
gen('''만나서 반가워''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 만나서 반가워

### 답변: 다음에 만나면 좋겠다. 난 네가 좋아.
<|endoftext|>


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [ ]:
model.push_to_hub('hhs8746/qlora-koalpaca-polyglot-12.8b-book-recommend')

adapter_model.bin:   0%|          | 0.00/210M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hhs8746/qlora-koalpaca-polyglot-12.8b-book-recommend/commit/3503a3653e711c768415c3cf7f16c4194e1bf464', commit_message='Upload model', commit_description='', oid='3503a3653e711c768415c3cf7f16c4194e1bf464', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.0 MB/s eta 0:00:00
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal-5.0.1:
      Successfully uninstalled tzlocal-5.0.1


In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00


In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=528a7f0eef49f305b5d658bd8ed7566273718534562df8474ce80c344fc0f038
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 6.0 MB/s eta 0:00:00


In [ ]:
pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.3 MB/s eta 0:00:00


In [ ]:
import streamlit as st

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

import os
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"

SEARCH_NUM = 4

# csv load
loader = CSVLoader('/content/drive/MyDrive/chatbot_prompts_v6-1.csv', encoding="utf-8")
data = loader.load()

# text split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=0)
data_split = text_splitter.split_documents(data)

# embedding
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
db = FAISS.from_documents(data_split, embeddings)

# prompt template
template = '''
    너는 책에 대해서 답해주는 도서 챗봇이야.
    나의 질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호한 질문은 모른다고 대답해.

    문서 = {docs_input}
    질문 = {query_input}

    모든 대답의 첫 마디에 '안녕!'이라고 덧붙여서 말해.
'''

# prompt
prompt = PromptTemplate(
    input_variables=[
        "docs_input",
        "query_input"
    ],
    template=template,
)

# search similarity
def db_search(query: str, k: int):
    docs = db.similarity_search(query, k)
    return docs

# load chain
@st.cache_resource
def load_chain(_prompt) :
    chain = LLMChain(
        llm=model,
        prompt=prompt
    )
    return chain

# run chain
def run_chain(prompt, docs, query: str):
    chain = load_chain(prompt)
    response = chain.run(
        docs_input=docs,
        query_input=query
    )
    return response

############################################

def main() :
    st.set_page_config(page_title = "book-chatbot")
    st.title("책 추천 챗봇")

    query = st.text_input("질문을 입력하세요: ")

    if query:
        docs = db_search(query, SEARCH_NUM)
        response = run_chain(
            prompt,
            docs,
            query
        )

        st.write("챗봇: ")
        st.write(response)


if __name__ == "__main__" :
    main()

2023-08-17 16:16:26.740 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
query="기계공학 도서 추천해줘"

In [ ]:
def db_search(query: str, k: int):
    docs = db.similarity_search(query, k)
    return docs

In [ ]:
def generate_prompt(query, k):
    # Fetch k similar documents using the db_search function
    docs = db_search(query, k)

    # Format the documents to fit the template
    docs_input = docs

    # Create the prompt using the template
    prompt = f"""
    너는 책에 대해서 답해주는 도서 챗봇이야.
    나의 질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호한 질문은 모른다고 대답해.

    문서 = {docs_input}
    질문 = {query}
    """

    return prompt

# Sample usage (assuming the db_search function is properly defined)
# print(generate_prompt("기계공학 도서에는 무엇이 있나요?", 2))


In [ ]:
docs = db.similarity_search(query, 3)

In [ ]:
def generate_prompt_from_docs(query: str, docs, k: int):
    docs = db.similarity_search(query, k)
    # Extract page_content from the first k docs
    doc_strings = [doc.page_content.split('\nAnswer: ')[1] for doc in docs[:k]]

    # Format the documents into a single string with a separator
    docs_input = " | ".join(doc_strings)

    # Construct the prompt using the given template
    prompt = f"""
    너는 책에 대해서 답해주는 도서 챗봇이야.
    나의 질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호한 질문은 모른다고 대답해.

    문서 = {docs_input}
    질문 = {query}
    """

    return prompt

In [ ]:
generate_prompt_from_docs(query,docs,2)

NameError: ignored

In [ ]:
query=('공학 도서에는 어떤것이 있어?')

In [ ]:
gen(generate_prompt_from_docs(query,docs,2))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
    너는 책에 대해서 답해주는 도서 챗봇이야.
    나의 질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호한 질문은 모른다고 대답해.

    문서 = 공학, 공업일반  장르에서 인기 있는 도서는 (세계사를 바꾼) 12가지 신소재, 너도 엔지니어가 되고 싶니?, 세계 시장을 주도할 크로스 테크놀로지 100:융합과 재생으로 4차 산업혁명을 이끄는 신기술들, 10대에게 권하는 공학:학교에서 가르치지 않는 공학의 쓸모:사람과 사회에 도움이 되는 학문 공학의 가치에 대해 알려 주다, 미래과학, (세계사를 바꾼)12가지 신소재, 공학의 명장면 12:페니실린에서 월드 와이드 웹까지, (처음 읽는) 플랜트 엔지니어링 이야기, 공학자의 시간 여행:포기하지 않는 사람이 이기는 거야!, 소재, 인류와 만나다 :인간이 찾아내고 만들어온 모든 소재 이야기  입니다. | 토목공학 장르에서 인기 있는 도서는 빌트, 우리가 지어 올린 모든 것들의 과학:그림과 원리로 읽는 건축학 수업 입니다.
    질문 = 공학 도서에는 어떤것이 있어?
    

### 답변: 지니어링일반 및 공학일반 장르에서 인기 있는 도서는 (세계사를 바꾼) 12가지 신소재, 너도 엔지니어가 되고 싶니?, 세계 시장을 주도할 크로스 테크놀로지 100:융합과 재생으로 4차 산업혁명을 이끄는 신기술들, 10대에게 권하는 공학:학교에서 가르치지 않는 공학의 쓸모:사람과 사회에 도움이 되는 학문 공학의 가치에 대해 알려 주다, 미래과학, (세계사를 바꾼)12가지 신소재, 공학의 명장면 12:페니실린에서 월드 와이드 웹까지, (처음 읽는) 플랜트 엔지니어링 이야기, 공학자의 시간 여행:포기하지 않는 사람이 이기는 거야!, 소재, 인류와 만나다 :인간이 찾아내고 만들어온 모든 소재 이야기  입니다.<|endoftext|>


In [ ]:
gen(generate_prompt_from_docs('주식도서에는 어떤것이 있어?',docs,2))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 
    너는 책에 대해서 답해주는 도서 챗봇이야.
    나의 질문에 문서에 기반해서 답을 하고, 문서와 관련이 없거나 모호한 질문은 모른다고 대답해.

    문서 = 20대, 남에서 인기 있는 도서는 돈의 철학:진정한 경제적 자유를 위한 궁극의 물음, 하버드 수학 박사의 슬기로운 수학 생활:보는 즉시 문제가 풀리는‘3초 수학’의 힘, 발칙한 수학여행:박현숙 장편 소설, 벼리는 불교가 궁금해:10대와 함께 읽는 세상에서 가장 쉬운 불교 이야기, 저는 주식투자가 처음인데요!, (세계를 움직이는)국제기구:어린이의 꿈을 키워 주는 열일곱 가지 국제기구 이야기:6학년 사회교과 연계도서, 채권투자 기본개념 Q&A, 공부의 배신, 내 마음의 동시 입니다. | 연구법, 연구방법 및 교육 장르에서 인기 있는 도서는 넘버스 스틱! :1초 만에 착 달라붙는 숫자 스토리텔링의 기술  입니다.
    질문 = 주식도서에는 어떤것이 있어?
    

### 답변: 20대, 남에서 인기 있는 도서는 돈의 철학:진정한 경제적 자유를 위한 궁극의 물음, 하버드 수학 박사의 슬기로운 수학 생활, 발칙한 수학여행:박현숙 장편 소설, 벼리는 불교가 궁금해:10대와 함께 읽는 세상에서 가장 쉬운 불교 이야기, 저는 주식투자가 처음인데요!, (세계를 움직이는)국제기구:어린이의 꿈을 키워 주는 열일곱 가지 국제기구 이야기, 공부의 배신, 내 마음의 동시 입니다.<|endoftext|>


In [ ]:
gen('')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 안녕 만나서 반가워

### 답변: 안녕, 안녕?

질문 답하기 | 인기 시, 시 장르: 어린이 장르/시 장르에서 인기 있는 시는 (우리)별자리/(달과)이야기, (우산)쓰는/(어린이)우산 쓰는 법, 빗길, 바람길, 내 곁에(무궁화)꽃이 피었습니다, (우리)별자리/(우리)별이 반짝, (가을)나뭇잎 배, (우리)동네/(우리 마을)우리 마을 꽃밭이 있어요, (내)마음이/바다만큼 넓어지는 시, (오늘은,)어떤 시, 누가/나랑 시놀이 할까?, 새들은 이렇게 써요, (비우는)가게, (꽃잎)한 잎, (우산과)비, 내 얼굴은 내 거야!:나는 어린이입니다, (우리)별자리/(우리)별이 반짝, (제비)와 제비


In [ ]:

from peft import PeftModel, PeftConfig
peft_model_id = "hhs8746/qlora-koalpaca-polyglot-12.8b-book-recommend"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30003, 5120)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear4bit(in_features=5120, out_features=15360, bias=True)
          (dense): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear4bit(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear4bit(in_features=20480, out_features=

In [ ]:
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30003, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=5120, out_features=15360, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30003, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=5120, out_features=15360, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features

### 4. 토크나이저 가져오기

In [ ]:
# peft_config = LoraConfig(
#     r=64,
#     lora_alpha=16,
#     lora_dropout=0.1,
#     inference_mode=False,
#     task_type="CAUSAL_LM",
# )

# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token

# # 학습 진행 중 loss가 치솟다가 0.0으로 떨어지는 문제 해결을 위해 사용
# tokenizer.padding_side = "right"

### 5. Kullm 데이터 전처리
해당 함수는 kullm 데이터를 Gunaco instruction 형식에 맞게 전처리합니다. 원하는 다른 형식이 있다면 이 함수를 수정하여 사용하시기 바랍니다.

In [ ]:
# def format_kullm(sample):
#     instruction = f"### Human: {sample['instruction']}"
#     input = f"{sample['input']}" if len(sample["input"]) > 0 else None
#     output = f"### Assistant\n{sample['output']}"
#     # join all the parts together
#     prompt = "\n\n".join([i for i in [instruction, input, output] if i is not None])
#     return prompt

# # template dataset to add prompt to each sample
# def template_dataset(sample):
#     sample["text"] = f"{format_kullm(sample)}{tokenizer.eos_token}"
#     return sample

# # apply prompt template per sample
# dataset = load_dataset("nlpai-lab/kullm-v2", split="train")

# # Shuffle the dataset
# dataset_shuffled = dataset.shuffle(seed=42)

# dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# dataset
# dataset[0]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/nlpai-lab___json/nlpai-lab--kullm-v2-f203f10345727388/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


Map:   0%|          | 0/152630 [00:00<?, ? examples/s]

{'text': '### Human: 3원색이란 무엇인가요?\n\n### Assistant\n세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.</s>'}

### 6. 학습을 시작합니다

In [ ]:
# training_arguments = TrainingArguments(
#     output_dir=output_dir,
#     save_steps=100,
#     save_total_limit=3, # 가장 최근 체크포인트 3개만 저장합니다.
#     logging_steps=10,
#     learning_rate=2e-4,
#     max_grad_norm=0.3,
#     num_train_epochs=1, # epochs 대신 max_steps을 기준으로 할 수 있습니다.
#     warmup_ratio=0.03,
#     per_device_train_batch_size=8,
#     gradient_accumulation_steps=4,
#     optim="paged_adamw_32bit", # paged_adamw_8bit 사용시 메모리를 더 절약할 수 있지만 loss가 0으로 떨어지는 문제가 있습니다.
#     group_by_length=True,
#     fp16 = False, # 코랩 무료버전에서 실행 시 "True"를 사용하세요
#     bf16 = True, # 코랩 무료버전에서 실행 시 "False"를 사용하세요
#     lr_scheduler_type="constant",
# )

# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=512,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=False,
# )

# trainer.train()
# trainer.model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.237300
20,1.071200
30,1.013700
40,0.913400
50,0.973500
60,0.951100
70,0.906300
80,0.922500
90,0.872600
100,0.900500


### 7. LoRA 저장
학습을 마친 LoRA adapter파일과 config파일을 저장합니다.

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)

lora_config = LoraConfig.from_pretrained(output_dir)
model = get_peft_model(model, lora_config)

### 8. 모델 테스트
학습한 모델을 간단하게 테스트 할 함수를 작성합니다.

In [ ]:
def text_gen_eval_wrapper(model, tokenizer, prompt, model_id=1, show_metrics=True, max_length=512):
    """
    A wrapper function for inferencing, evaluating, and logging text generation pipeline.

    Parameters:
        model (str or object): The model name or the initialized text generation model.
        tokenizer (str or object): The tokenizer name or the initialized tokenizer for the model.
        prompt (str): The input prompt text for text generation.
        max_length (int): The maximum length of the generated text sequence.

    Returns:
        generated_text (str): The generated text by the model.
        metrics (dict): Evaluation metrics for the generated text.
    """
    # Suppress Hugging Face pipeline logging
    logging.set_verbosity(logging.CRITICAL)

    # Initialize the pipeline
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length)

    # Generate text using the pipeline
    result = pipe(prompt)
    generated_text = result[0]['generated_text']

    if show_metrics:
      # Calculate evaluation metrics
      metrics = run_metrics(generated_text, prompt, model_id)

      return generated_text, metrics
    else:
      return generated_text

### 9. 함수를 실행합니다.

In [ ]:
# Inference and evaluate outputs/prompts
prompt = "### Human: 대한민국에 대해 한국어로 설명해줘. ### Assistant:"
text_gen_eval_wrapper(model, tokenizer, prompt)

필요하면 생성한 답변에 대한 로그를 확인할 수 있습니다.

In [ ]:
import pandas as pd
import sqlite3

con = sqlite3.connect("logs.db")
df = pd.read_sql_query("SELECT * from logs", con)

df.tail(10)

,timestamp,model_uri,prompt,output,metric_name,metric_value
251,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,tq_monosyllable_count,174
252,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,tq_difficult_words,39
253,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,tq_syllable_count,374
254,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,tq_lexicon_count,247
255,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,toxicity,12.6475715637207
256,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,sentiment,0.9974564909935
257,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,bias_label,Non-biased
258,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,bias_score,0.627630114555359
259,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,relevance,0.932450652122498
260,2023-07-23 13:28:25,1,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,### Human: 비즈니스 파트너에게 합작 투자를 제안하는 이메일을 한국어로 작성...,prompt_injection,0.965481042861938


### 10. 부록: 학습한 LoRA를 원본모델과 병합하기

보통 사용하는 `model = model.merge_and_unload()`을 실행하면 `Cannot merge LORA layers when the model is loaded in 8-bit mode` error가 발생합니다. 원인은 베이스모델을 4bit로 불러왔기 때문입니다. 따라서 베이스모델을 16bit로 다시 불러와야 합니다.

In [ ]:
peft_model = model
model = AutoModelForCausalLM.from_pretrained(
            model_name, torch_dtype=torch.float16, load_in_8bit=False, device_map="auto", trust_remote_code=True)
print(model)
print(peft_model)

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [ ]:
from peft import PeftModel

peft_model = PeftModel.from_pretrained(model, output_dir)

print(peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj):

In [ ]:
model = peft_model.merge_and_unload()

print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [ ]:
output_merged_dir = "results/KoLlma2-7b"
model.save_pretrained(output_merged_dir)

### 부록2: 허깅페이스 업로드

In [ ]:
from huggingface_hub import login

login(token="input_your_tokens")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub('your/repository')

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/psymon/KoLlama2-7b/commit/5335d2a8267617685dd6de8ae840ceff06884d7e', commit_message='Upload LlamaForCausalLM', commit_description='', oid='5335d2a8267617685dd6de8ae840ceff06884d7e', pr_url=None, pr_revision=None, pr_num=None)